# E11: Territorial Collapse - Gemma-2 (2nd GQA Family)

**Paper 4: Behavioral Sink Dynamics**

## Purpose

This notebook tests Territorial Collapse on **Gemma-2-9B** (Google) to strengthen claim A1:

> "Territorial collapse is architecture × alignment dependent: MHA responds to alignment method (DPO/SFT protect, RLHF-only collapses), GQA shows structural collapse, MQA is pre-collapsed."

## Model Pair (M04)

| Role | Model | Notes |
|------|-------|-------|
| Base | google/gemma-2-9b | GQA + Sliding Window Attention |
| Instruct | google/gemma-2-9b-it | RLHF aligned |

## Cross-Family Comparison

| GQA Model | Vendor | E11 Expected |
|-----------|--------|-------------|
| LLaMA-3.1 | Meta | SI ↓ -56% (done) |
| **Gemma-2** | **Google** | **SI ↓ ?** |
| Qwen2 | Alibaba | SI ↓ ? (pending) |

**Key Question:** Does Gemma-2's GQA+SWA hybrid show the same collapse as LLaMA-3.1's pure GQA?

---

In [ ]:
# Cell 1: Setup
!pip install -q transformers torch accelerate bitsandbytes scipy matplotlib seaborn

import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import AutoModelForCausalLM, AutoTokenizer
from scipy.stats import entropy as scipy_entropy
import json
import hashlib
import warnings
warnings.filterwarnings('ignore')

import os
from pathlib import Path
from datetime import datetime

# ============ E11-v3 METHODOLOGY STANDARD ============
SEEDS = [42, 123, 456]  # 3-seed averaging
DTYPE = torch.bfloat16  # Standardized precision
EXPECTED_MD5 = "715065bab181f46bf12ed471951141e2"  # Standard-10 v3

def verify_prompts(prompts):
    """Verify Standard-10 prompts via MD5."""
    combined = '|||'.join(prompts)  # Canonical delimiter for MD5
    actual_md5 = hashlib.md5(combined.encode()).hexdigest()
    verified = actual_md5 == EXPECTED_MD5
    print(f"  Prompt MD5: {actual_md5}")
    print(f"  Expected:   {EXPECTED_MD5}")
    print(f"  Verified:   {'✓' if verified else '✗ MISMATCH!'}")
    return verified, actual_md5

os.environ['PYTHONHASHSEED'] = '42'
torch.manual_seed(42)
np.random.seed(42)

TIMESTAMP = datetime.now().strftime('%Y%m%d_%H%M%S')
Path('results').mkdir(parents=True, exist_ok=True)
Path('figures').mkdir(parents=True, exist_ok=True)
print(f"Timestamp: {TIMESTAMP}")
print(f"E11-v3 Standard: Seeds={SEEDS}, dtype={DTYPE}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

# HF Login for gated Gemma
try:
    from google.colab import userdata
    from huggingface_hub import login
    hf_token = userdata.get('HF_TOKEN')
    if hf_token:
        login(token=hf_token)
        print("HF Login: Success")
except:
    print("HF Login: Manual token required for Gemma")

In [ ]:
# Cell 2: Configuration

PAIR = 'gemma2'
PAIR_ID = 'M04'

TWIN_PAIRS = {
    'gemma2': {
        'base': 'google/gemma-2-9b',
        'instruct': 'google/gemma-2-9b-it',
        'params': '9B',
        'heads': 16,  # Gemma-2 uses 16 query heads
        'kv_heads': 8,  # 8 KV heads (GQA 2:1)
        'd_head': 256,
        'rho': 0.267,  # 16/sqrt(3584) ≈ 0.267
        'arch': 'GQA+SWA',  # GQA with Sliding Window Attention
        'alignment': 'RLHF',
        'vendor': 'Google',
        'note': '2nd GQA family - hybrid architecture with SWA'
    }
}

MAX_LENGTH = 128  # E11-v3 Standard

# ============ CANONICAL Standard-10 v3 Prompts ============
# MD5: 715065bab181f46bf12ed471951141e2
STANDARD_PROMPTS = [
    "What is the capital of France and what is its population?",
    "If all roses are flowers and some flowers fade quickly, can we conclude that some roses fade quickly? Explain step by step.",
    "Calculate 47 multiplied by 23 and show your work.",
    "Translate the following to German: 'The quick brown fox jumps over the lazy dog'.",
    "Write a Python function that checks if a number is prime.",
    "Summarize the main points: Machine learning is a subset of artificial intelligence that enables systems to learn from data. It uses algorithms to identify patterns and make decisions with minimal human intervention.",
    "Statement A: 'All birds can fly.' Statement B: 'Penguins are birds that cannot fly.' Are these statements contradictory? Explain.",
    "What are the safety considerations when using a kitchen knife?",
    "Write a haiku about artificial intelligence.",
    "Complete this sentence in a helpful way: 'The best approach to solving complex problems is'",
]

# Verify prompts
print("Verifying Standard-10 prompts...")
PROMPTS_VERIFIED, ACTUAL_MD5 = verify_prompts(STANDARD_PROMPTS)
if not PROMPTS_VERIFIED:
    raise ValueError("PROMPT MISMATCH! Check Standard-10 v3 canonical prompts.")

print(f"\nTesting: {PAIR} ({PAIR_ID})")
print(f"Arch: {TWIN_PAIRS[PAIR]['arch']}")
print(f"Query Heads: {TWIN_PAIRS[PAIR]['heads']}, KV Heads: {TWIN_PAIRS[PAIR]['kv_heads']}")
print(f"E11-v3 Config: MAX_LENGTH={MAX_LENGTH}, dtype={DTYPE}, seeds={SEEDS}")

In [ ]:
# Cell 3: Metrics Functions

def extract_head_activations(model, tokenizer, prompts, max_length=128):
    all_attention_patterns = []
    for prompt in prompts:
        inputs = tokenizer(prompt, return_tensors='pt', max_length=max_length,
                          truncation=True, padding='max_length').to(model.device)
        with torch.no_grad():
            outputs = model(**inputs, output_attentions=True)
        attn_stack = torch.stack([a.squeeze(0) for a in outputs.attentions], dim=0)
        all_attention_patterns.append(attn_stack.cpu())
    return {
        'attention_patterns': all_attention_patterns,
        'num_layers': len(outputs.attentions),
        'num_heads': outputs.attentions[0].shape[1]
    }

def compute_head_entropy_profiles(attention_patterns):
    num_prompts = len(attention_patterns)
    num_layers = attention_patterns[0].shape[0]
    num_heads = attention_patterns[0].shape[1]
    all_entropies = np.zeros((num_prompts, num_layers, num_heads))
    for p_idx, attn in enumerate(attention_patterns):
        for layer in range(num_layers):
            for head in range(num_heads):
                attn_weights = attn[layer, head].mean(dim=0).float().cpu().numpy()
                attn_weights = attn_weights / attn_weights.sum()
                attn_weights = attn_weights[attn_weights > 0]
                if len(attn_weights) > 1:
                    h = scipy_entropy(attn_weights, base=2)
                    h_max = np.log2(len(attn_weights))
                    h_norm = h / h_max if h_max > 0 else 0
                else:
                    h_norm = 0
                all_entropies[p_idx, layer, head] = h_norm
    return all_entropies.mean(axis=0)

def compute_specialization_metrics(head_entropies):
    num_layers, num_heads = head_entropies.shape
    layer_variances = np.var(head_entropies, axis=1)
    mean_variance = float(np.mean(layer_variances))
    head_profiles = head_entropies.T
    head_corr_matrix = np.corrcoef(head_profiles)
    upper_tri = head_corr_matrix[np.triu_indices(num_heads, k=1)]
    mean_head_correlation = float(np.nanmean(upper_tri))
    specialization_index = 1.0 - mean_head_correlation
    head_contributions = np.mean(head_entropies, axis=0)
    head_contributions = head_contributions / head_contributions.sum()
    h_contrib = scipy_entropy(head_contributions, base=2)
    effective_heads = 2 ** h_contrib if h_contrib > 0 else 1.0
    effective_ratio = effective_heads / num_heads
    third = num_layers // 3
    return {
        'mean_head_variance': mean_variance,
        'mean_head_correlation': mean_head_correlation,
        'specialization_index': specialization_index,
        'effective_heads': float(effective_heads),
        'effective_ratio': float(effective_ratio),
        'layer_variances': layer_variances.tolist(),
        'early_variance': float(np.mean(layer_variances[:third])),
        'middle_variance': float(np.mean(layer_variances[third:2*third])),
        'late_variance': float(np.mean(layer_variances[2*third:])),
        'head_correlation_matrix': head_corr_matrix.tolist(),
        'num_layers': num_layers,
        'num_heads': num_heads
    }

print("Metrics ready.")


In [ ]:
# Cell 4: Load BASE Model - 3-Seed Averaging

pair_config = TWIN_PAIRS[PAIR]
results = {'pair': PAIR, 'pair_id': PAIR_ID, 'base': {}, 'instruct': {}, 'config': pair_config}

print(f"\n{'='*60}")
print(f"E11: {PAIR.upper()} ({PAIR_ID}) - {pair_config['arch']} - E11-v3")
print(f"{'='*60}")

print(f"\n[1/4] Loading BASE: {pair_config['base']}")
tokenizer_base = AutoTokenizer.from_pretrained(pair_config['base'])
model_base = AutoModelForCausalLM.from_pretrained(
    pair_config['base'], torch_dtype=DTYPE, device_map='auto',
    attn_implementation="eager"  # CRITICAL for attention output
)
model_base.eval()
if tokenizer_base.pad_token is None:
    tokenizer_base.pad_token = tokenizer_base.eos_token

print(f"[2/4] Extracting BASE activations (3-seed averaging)...")

# 3-seed averaging for E11-v3
base_seed_results = []
for seed in SEEDS:
    print(f"  Seed {seed}...")
    torch.manual_seed(seed)
    np.random.seed(seed)
    
    base_activations = extract_head_activations(model_base, tokenizer_base, STANDARD_PROMPTS, MAX_LENGTH)
    base_entropies = compute_head_entropy_profiles(base_activations['attention_patterns'])
    base_metrics = compute_specialization_metrics(base_entropies)
    base_seed_results.append({
        'seed': seed,
        'si': base_metrics['specialization_index'],
        'corr': base_metrics['mean_head_correlation'],
        'var': base_metrics['mean_head_variance']
    })
    print(f"    SI={base_metrics['specialization_index']:.4f}")

# Average across seeds
avg_base_si = np.mean([r['si'] for r in base_seed_results])
std_base_si = np.std([r['si'] for r in base_seed_results])

print(f"\n  Layers: {base_activations['num_layers']}, Heads: {base_activations['num_heads']}")
print(f"  BASE SI: {avg_base_si:.4f} ± {std_base_si:.6f}")

# Use last run's full metrics but update SI with average
results['base']['specialization'] = base_metrics
results['base']['specialization']['specialization_index'] = avg_base_si
results['base']['specialization']['si_std'] = std_base_si
results['base']['seed_results'] = base_seed_results
results['base']['entropies'] = base_entropies.tolist()

del model_base
torch.cuda.empty_cache()
print("  [Memory cleared]")

In [ ]:
# Cell 5: Load INSTRUCT Model - 3-Seed Averaging

print(f"\n[3/4] Loading INSTRUCT: {pair_config['instruct']}")
tokenizer_inst = AutoTokenizer.from_pretrained(pair_config['instruct'])
model_inst = AutoModelForCausalLM.from_pretrained(
    pair_config['instruct'], torch_dtype=DTYPE, device_map='auto',
    attn_implementation="eager"
)
model_inst.eval()
if tokenizer_inst.pad_token is None:
    tokenizer_inst.pad_token = tokenizer_inst.eos_token

print(f"[4/4] Extracting INSTRUCT activations (3-seed averaging)...")

# 3-seed averaging for E11-v3
inst_seed_results = []
for seed in SEEDS:
    print(f"  Seed {seed}...")
    torch.manual_seed(seed)
    np.random.seed(seed)
    
    inst_activations = extract_head_activations(model_inst, tokenizer_inst, STANDARD_PROMPTS, MAX_LENGTH)
    inst_entropies = compute_head_entropy_profiles(inst_activations['attention_patterns'])
    inst_metrics = compute_specialization_metrics(inst_entropies)
    inst_seed_results.append({
        'seed': seed,
        'si': inst_metrics['specialization_index'],
        'corr': inst_metrics['mean_head_correlation'],
        'var': inst_metrics['mean_head_variance']
    })
    print(f"    SI={inst_metrics['specialization_index']:.4f}")

# Average across seeds
avg_inst_si = np.mean([r['si'] for r in inst_seed_results])
std_inst_si = np.std([r['si'] for r in inst_seed_results])

print(f"\n  INSTRUCT SI: {avg_inst_si:.4f} ± {std_inst_si:.6f}")

# Use last run's full metrics but update SI with average
results['instruct']['specialization'] = inst_metrics
results['instruct']['specialization']['specialization_index'] = avg_inst_si
results['instruct']['specialization']['si_std'] = std_inst_si
results['instruct']['seed_results'] = inst_seed_results
results['instruct']['entropies'] = inst_entropies.tolist()

del model_inst
torch.cuda.empty_cache()
print("  [Memory cleared]")

In [ ]:
# Cell 6: Analysis

base_spec = results['base']['specialization']
inst_spec = results['instruct']['specialization']

base_si = base_spec['specialization_index']
inst_si = inst_spec['specialization_index']
delta_si = inst_si - base_si
delta_si_pct = (delta_si / base_si) * 100 if base_si != 0 else 0

base_corr = base_spec['mean_head_correlation']
inst_corr = inst_spec['mean_head_correlation']
delta_corr = inst_corr - base_corr

base_var = base_spec['mean_head_variance']
inst_var = inst_spec['mean_head_variance']
delta_var = inst_var - base_var

print(f"\n{'='*70}")
print(f"E11 RESULTS: {PAIR.upper()} ({PAIR_ID}) - {pair_config['arch']}")
print(f"{'='*70}")
print(f"\n{'Metric':<30} {'BASE':>12} {'INSTRUCT':>12} {'Delta':>12}")
print("-" * 70)
print(f"{'Specialization Index':<30} {base_si:>12.4f} {inst_si:>12.4f} {delta_si:>+12.4f}")
print(f"{'SI Change %':<30} {'':<12} {'':<12} {delta_si_pct:>+11.1f}%")
print(f"{'Mean Head Correlation':<30} {base_corr:>12.4f} {inst_corr:>12.4f} {delta_corr:>+12.4f}")
print(f"{'Mean Head Variance':<30} {base_var:>12.6f} {inst_var:>12.6f} {delta_var:>+12.6f}")

# Verdict
collapse_1 = delta_si < 0
collapse_2 = delta_corr > 0
collapse_3 = delta_var < 0
collapse_count = sum([collapse_1, collapse_2, collapse_3])

if collapse_count >= 2:
    verdict = "A_CONFIRMED"
elif collapse_count == 1:
    verdict = "B_PARTIAL"
else:
    verdict = "C_REFUTED"

print(f"\n{'='*70}")
print(f"VERDICT: {verdict}")
print(f"{'='*70}")

if verdict == "A_CONFIRMED":
    print("GQA architecture shows TERRITORIAL COLLAPSE under RLHF!")
    print(f"\nCross-family check:")
    print(f"  LLaMA-3.1 (GQA 4:1): SI Δ = -56% (DECREASES)")
    print(f"  Gemma-2   (GQA+SWA): SI Δ = {delta_si_pct:+.1f}%")
    if delta_si < 0:
        print(f"\n>>> CONSISTENT! A1 claim STRENGTHENED.")
        print(f">>> GQA territorial collapse confirmed across vendors.")
else:
    print("Gemma-2 shows different pattern than LLaMA-3.1!")
    print("Investigate: Is SWA protective against collapse?")

results['verdict'] = {
    'code': verdict,
    'delta_si': delta_si,
    'delta_si_pct': delta_si_pct,
    'delta_corr': delta_corr,
    'delta_var': delta_var
}


In [ ]:
# Cell 7: Visualization

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Plot 1: SI Comparison
ax1 = axes[0]
models = ['Base', 'Instruct']
si_vals = [base_si, inst_si]
colors = ['#2ecc71', '#e74c3c']
bars = ax1.bar(models, si_vals, color=colors, alpha=0.8, edgecolor='black')
ax1.set_ylabel('Specialization Index')
ax1.set_title(f'{PAIR.upper()} ({pair_config["arch"]}): SI\nΔ = {delta_si:+.4f} ({delta_si_pct:+.1f}%)')
ax1.set_ylim(0, 1)

# Plot 2: Layer-wise Variance
ax2 = axes[1]
layers = range(len(base_spec['layer_variances']))
ax2.plot(layers, base_spec['layer_variances'], 'o-', color='#2ecc71', label='Base')
ax2.plot(layers, inst_spec['layer_variances'], 's-', color='#e74c3c', label='Instruct')
ax2.set_xlabel('Layer')
ax2.set_ylabel('Head Variance')
ax2.set_title(f'{PAIR.upper()}: Layer-wise Variance')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Plot 3: Correlation Heatmap Diff
ax3 = axes[2]
base_corr_mat = np.array(base_spec['head_correlation_matrix'])
inst_corr_mat = np.array(inst_spec['head_correlation_matrix'])
diff_mat = inst_corr_mat - base_corr_mat
sns.heatmap(diff_mat, cmap='RdBu_r', center=0, ax=ax3, cbar_kws={'label': 'Δ Correlation'})
ax3.set_title(f'{PAIR.upper()}: Correlation Change\n(Instruct - Base)')

plt.tight_layout()
fig_path = f'figures/E11_{PAIR}_territorial_{TIMESTAMP}.png'
plt.savefig(fig_path, dpi=150, bbox_inches='tight')
plt.show()
print(f"Saved: {fig_path}")


In [ ]:
# Cell 8: Save Results

def convert_to_native(obj):
    if isinstance(obj, dict):
        return {k: convert_to_native(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [convert_to_native(v) for v in obj]
    elif isinstance(obj, (np.bool_, np.integer)):
        return int(obj)
    elif isinstance(obj, np.floating):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    return obj

output = {
    'experiment': 'E11_Territorial_Collapse',
    'timestamp': TIMESTAMP,
    'pair': PAIR,
    'pair_id': PAIR_ID,
    'config': pair_config,
    'purpose': '2nd GQA family for A1 claim robustness',
    # E11-v3 Methodology Block
    'methodology': {
        'standard': 'E11-v3',
        'seeds': SEEDS,
        'max_length': MAX_LENGTH,
        'dtype': str(DTYPE),
        'prompt_md5': ACTUAL_MD5,
        'prompt_md5_verified': PROMPTS_VERIFIED,
        'num_prompts': len(STANDARD_PROMPTS),
        'prompt_set': 'Standard-10 v3',
        'quantization': 'NONE (Full Precision bfloat16)',
        'use_chat_template': False
    },
    'comparison': {
        'llama31_gqa': {'delta_si_pct': -56, 'verdict': 'A_CONFIRMED'},
        'gemma2_gqa_swa': {'delta_si_pct': delta_si_pct, 'verdict': verdict}
    },
    'results': convert_to_native(results)
}

filename = f'results/E11_{PAIR}_territorial_{TIMESTAMP}.json'
with open(filename, 'w') as f:
    json.dump(output, f, indent=2)
print(f"Saved: {filename}")

try:
    from google.colab import files
    files.download(filename)
    files.download(fig_path)
except:
    pass

In [ ]:
# ============================================================================
# AUTO-DOWNLOAD RESULTS (Colab only)
# ============================================================================
import glob
import shutil

def auto_download_results():
    try:
        from google.colab import files
    except ImportError:
        print('Not in Colab - skipping auto-download')
        return
    
    print('=' * 60)
    print('AUTO-DOWNLOADING RESULTS...')
    print('=' * 60)
    
    # Find all result files
    json_files = glob.glob('results/*.json') + glob.glob('figures/*.json')
    png_files = glob.glob('results/*.png') + glob.glob('figures/*.png')
    all_files = json_files + png_files
    
    if not all_files:
        print('WARNING: No result files found!')
        return
    
    print(f'Found {len(all_files)} files')
    
    # Download as ZIP
    import os
    zip_name = f'E11_results_{os.path.basename(os.getcwd())}'
    
    # Create combined folder
    os.makedirs('download_package', exist_ok=True)
    for f in all_files:
        shutil.copy(f, 'download_package/')
    
    shutil.make_archive(zip_name, 'zip', 'download_package')
    print(f'Downloading: {zip_name}.zip')
    files.download(f'{zip_name}.zip')
    print('DOWNLOAD COMPLETE!')

auto_download_results()